# Artikel-Update bei Chip-Teilen in Count-IT DB

## 1\. Export data

Run on: Main AdvanTex DB (SALADVPSQLC1A1.salres.com)  
Extract all Chips for articles in use at specific customers.  
Save the result as .csv file

In [ ]:
SELECT OPTeile.Code, Artikel.ArtikelNr
FROM Salesianer.dbo.OPTeile
JOIN Salesianer.dbo.Artikel ON OPTeile.ArtikelID = Artikel.ID
WHERE Artikel.ID IN (
    SELECT DISTINCT KdArti.ArtikelID
    FROM Salesianer.dbo.KdArti
    JOIN Salesianer.dbo.Kunden ON KdArti.KundenID = Kunden.ID
    WHERE Kunden.KdNr IN (10002413, 130042, 10001006, 10001137, 10001365, 10001367, 10001370, 212066, 10001363, 10004548, 10004549, 232257)
  )
  AND LEN(OPTeile.Code) = 24;

GO

## 2\. Import Data

Import the data from the first script (from the saved .csv-file) into a table named \_\_AdvUHF in the LaundryAutomation DB on the Count-IT DB Server (SRVATENCITDB01)  
Table structure should be: Code nchar(24), ArtikelNr nchar(15)

## 3\. Update chips

Run on: Count-IT DB (SRVATENCITDB01)

Updates all chips where the article in AdvanTex does not match the article in the Count-IT DB (Table: SalesianerChip)

In [ ]:
WITH CurrentArticle AS (
  SELECT a.ArticleID, a.ArticleNumber
  FROM LaundryAutomation.dbo.Article AS a
  WHERE a.LastTransmissionDate >= DATEADD(day, -7, GETDATE())
)
UPDATE LaundryAutomation.dbo.SalesianerChip SET ArticleID = CurrentArticle.ArticleID
FROM LaundryAtomation.dbo.SalesianerChip
JOIN LaundryAutomation.dbo.__AdvUHF ON __AdvUHF.Code = SalesianerChip.Sgtin96HexCode
JOIN CurrentArticle ON __AdvUHF.ArtikelNr = CurrentArticle.ArticleNumber
WHERE SalesianerChip.ArticleID != CurrentArticle.ArticleID;